In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import cross_val_score
from joblib import Parallel, delayed
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('.\Datasets\CR_COST_FC.csv', converters={'PARTY_ID': str, 'COM_ID': str, 'CNTR_SIZE': str})
df['POD'] = pd.to_datetime(df['POD'])
df['ENCODED_TYPE'] = df['ENCODED_TYPE'].fillna(-1).astype(int)
df = df.dropna(subset=['ENCODED_TYPE'])
df['RATE'] = df['RATE'].fillna(-1).astype(int)
df = df.dropna(subset=['RATE'])
df['ENCODED_TYPE'] = df['ENCODED_TYPE'].astype(int)

# Remove NAN values
df_clean= df.dropna().reset_index(drop=True)

df_clean.head()
df_clean.info()
print(f'Dataset size: {df_clean.shape}')

,COM_ID,PARTY_ID,PARTY_ID_EN,PARTY_NAME,CSL_ID,ETA_ETD_NO,ETD_POL_D,POD_ID,POD,CNTR_ID,CNTR_SIZE,CNTR_TYPE,COST_TERM,RATE,ENCODED_TYPE,ENCODED_TERM,POD_ID_EN
0,10,10003678,10003678,OY TAVATUR SHIPPING LTD,ECS010060100191,20060228,FIHEL,HELSINKI,2006-02-28 00:00:00.0,CTNR010060201232,20,GP,FC,675.0,2.0,1,4.103561e+09
1,10,10004286,10004286,"GLOBELINK KOREA CO., LTD",ECS010060100465,20060303,KRPUS,BUSAN,2006-03-03 00:00:00.0,CTNR010060300014,40,HC,FC,390.0,1.0,1,2.904902e+09
2,10,1005136,1005136,MONDIALE FREIGHT SERVICES LIMITED-AUCKLAND,ECS010060100321,20060227,NZAKL,AUCKLAND,2006-02-27 00:00:00.0,CTNR010060201198,40,HC NOR,FC,1615.0,4.0,1,2.397848e+09
3,10,1002775,1002775,GLOBELINK INTERNATIONAL PTY LTD-ADELAIDE,ECS010060200201,20060306,AUADL,ADELAIDE,2006-03-06 00:00:00.0,CTNR010060300119,40,HC,FC,700.0,1.0,1,2.480737e+08
4,10,1001088,1001088,CARGO LINES S.A.-LE HAVRE,ECS010060200146,20060307,FRMRS,MARSEILLE,2006-03-07 00:00:00.0,CTNR010060300185,20,GP,FC,675.0,2.0,1,4.197227e+08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54089 entries, 0 to 54088
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   COM_ID        54089 non-null  int64  
 1   PARTY_ID      54089 non-null  int64  
 2   PARTY_ID_EN   54089 non-null  int64  
 3   PARTY_NAME    54089 non-null  object 
 4   CSL_ID        54089 non-null  object 
 5   ETA_ETD_NO    54089 non-null  int64  
 6   ETD_POL_D     54089 non-null  object 
 7   POD_ID        54083 non-null  object 
 8   POD           54089 non-null  object 
 9   CNTR_ID       54089 non-null  object 
 10  CNTR_SIZE     54089 non-null  int64  
 11  CNTR_TYPE     54089 non-null  object 
 12  COST_TERM     54089 non-null  object 
 13  RATE          54089 non-null  float64
 14  ENCODED_TYPE  54062 non-null  float64
 15  ENCODED_TERM  54089 non-null  int64  
 16  POD_ID_EN     54083 non-null  float64
dtypes: float64(3), int64(6), object(8)
memory usage: 7.0+ MB
Dataset size